In [1]:
import pandas as pd

def format_timestamp(ts):
    """Return timestamp as 'YYYY-MM-DD HH:MM:SS.ffffff Z' and normalize to UTC.

    Prefers a broker-provided ISO 'time' string (which may include 'Z' for UTC).
    If given a naive datetime, assumes UTC (change to localize differently if your broker docs say otherwise).
    """
    if ts is None:
        return str(ts)
    try:
        # If caller passed the whole price dict, prefer its 'time' ISO string
        if isinstance(ts, dict):
            iso = ts.get('time') or ts.get('timestamp')
            if iso:
                parsed = pd.to_datetime(iso, utc=True)
                return parsed.strftime('%Y-%m-%d %H:%M:%S.%f %Z')

        # If it's a string, parse with UTC awareness (handles trailing 'Z')
        if isinstance(ts, str):
            parsed = pd.to_datetime(ts, utc=True)
            return parsed.strftime('%Y-%m-%d %H:%M:%S.%f %Z')

        # If it's a datetime-like object
        if hasattr(ts, 'tzinfo'):
            # pd.Timestamp will preserve tz if present; if naive, localize to UTC
            ts_ts = pd.Timestamp(ts)
            if ts_ts.tzinfo is None:
                ts_ts = ts_ts.tz_localize('UTC')
            else:
                ts_ts = ts_ts.tz_convert('UTC')
            return ts_ts.strftime('%Y-%m-%d %H:%M:%S.%f %Z')

        # Fallback to pandas parsing (makes result UTC-aware)
        parsed = pd.to_datetime(ts, utc=True)
        return parsed.strftime('%Y-%m-%d %H:%M:%S.%f %Z')
    except Exception:
        return str(ts)

In [2]:
import datetime
format_timestamp(datetime.datetime(2025, 9, 5, 13, 28, 10, 661170),)  # Example usage

'2025-09-05 13:28:10.661170 UTC'

In [7]:
# ...existing code...
import calendar
import datetime

def to_millis(dt, assume_utc=True):
    """Convert datetime -> int milliseconds since epoch (UTC).
    Uses integer arithmetic (calendar.timegm) to avoid float timestamp() cost.
    Truncates microseconds to millisecond precision.
    """
    # If naive, assume UTC (change if broker docs say otherwise)
    if dt.tzinfo is None and assume_utc:
        seconds = calendar.timegm(dt.timetuple())
    else:
        # Make UTC-aware and use timegm on UTC timetuple
        dt_utc = dt.astimezone(datetime.timezone.utc) if dt.tzinfo is not None else dt
        seconds = calendar.timegm(dt_utc.timetuple())
    return seconds * 1000 + (dt.microsecond // 1000)

def from_millis(ms):
    """Convert int milliseconds since epoch -> datetime.datetime (UTC, tz-aware)."""
    import datetime
    ms = int(ms)
    seconds = ms // 1000
    ms_rem = ms % 1000
    dt = datetime.datetime.fromtimestamp(seconds, tz=datetime.timezone.utc)
    return dt.replace(microsecond=ms_rem * 1000)

# Example
dt = datetime.datetime(2025, 9, 5, 13, 28, 10, 661170)
print(to_millis(dt))  # -> integer milliseconds since epoch

1757078890661


In [9]:
from_millis(1757078890661)  # Example usage

datetime.datetime(2025, 9, 5, 13, 28, 10, 661000, tzinfo=datetime.timezone.utc)